In [5]:
import statsmodels.api as sm
import pandas as pd

In [3]:
results = pd.read_csv('/code/experiments/export/wandb_export_2024-01-06T22_35_05.940-08_00.csv')
results.head()

,Name,State,Notes,User,Tags,Created,Runtime,Sweep,activation_type,architecture,...,output_size,best_epoch,best_validation_loss,epoch,step_learning_rate,step_training_loss,step_validation_loss,weighted_f1,weighted_precision,weighted_recall
0,peach-sweep-41,finished,-,shane-kercheval,NaN,2024-01-07T03:41:18.000Z,357,vy3v7jae,leaky_relu,CNN,...,NaN,20.0,0.027257,26.0,0.000031,0.001481,0.027563,0.993428,0.993441,0.993429
1,misunderstood-sweep-31,finished,-,shane-kercheval,NaN,2024-01-07T01:07:22.000Z,2291,vy3v7jae,leaky_relu,CNN,...,NaN,22.0,0.027705,28.0,0.000313,0.009203,0.027450,0.992860,0.992882,0.992857
2,proud-sweep-9,finished,-,shane-kercheval,NaN,2024-01-06T21:32:02.000Z,569,vy3v7jae,relu,CNN,...,NaN,16.0,0.028781,28.0,0.000031,0.005160,0.031433,0.991576,0.991595,0.991571
3,balmy-sweep-33,finished,-,shane-kercheval,NaN,2024-01-07T02:01:27.000Z,343,vy3v7jae,leaky_relu,CNN,...,NaN,11.0,0.027821,26.0,0.000313,0.000551,0.032773,0.993857,0.993867,0.993857
4,generous-sweep-48,finished,-,shane-kercheval,NaN,2024-01-07T04:45:17.000Z,916,vy3v7jae,leaky_relu,CNN,...,NaN,13.0,0.034519,22.0,0.000313,0.003516,0.038947,0.994285,0.994291,0.994286


In [4]:
results.iloc[0].transpose()

Name                                   peach-sweep-41
State                                        finished
Notes                                               -
User                                  shane-kercheval
Tags                                              NaN
Created                      2024-01-07T03:41:18.000Z
Runtime                                           357
Sweep                                        vy3v7jae
activation_type                            leaky_relu
architecture                                      CNN
batch_size                                         64
conv_dropout_p                                    0.2
device                                           cuda
early_stopping_delta                             0.05
early_stopping_delta_type                    relative
early_stopping_patience                             3
epochs                                            100
accuracy                                     0.993429
test_loss                   

In [60]:
from sklearn.preprocessing import StandardScaler

data = results[results['test_loss'].notna()]
X = data[[
    'use_batch_norm',
    'optimizer',
    'batch_size',
    'include_second_fc_layer',
    'activation_type',
    'fc_dropout_p',
    'kernel_sizes',
    'out_channels',
]]
# Separate numeric and non-numeric columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
print(f"numeric_cols: {list(numeric_cols)}")
non_numeric_cols = X.select_dtypes(exclude=['float64', 'int64']).columns
print(f"non_numeric_cols: {list(non_numeric_cols)}")

# Standardize numeric columns
scaler = StandardScaler()
X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
# Convert non-numeric columns to dummy variables
X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)
X = X.replace({True: 1, False: 0})
# X = X.to_numpy()
y = data['test_loss']
# standardize the response variable
scaler = StandardScaler()
y = scaler.fit_transform(y.to_numpy().reshape(-1, 1))
# Add a constant to the model (intercept)
X = sm.add_constant(X)
# Fit the logistic regression model
model = sm.OLS(y, X).fit()
# Print the summary
print(model.summary())

numeric_cols: ['batch_size', 'fc_dropout_p']
non_numeric_cols: ['use_batch_norm', 'optimizer', 'include_second_fc_layer', 'activation_type', 'kernel_sizes', 'out_channels']
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     2.078
Date:                Sun, 07 Jan 2024   Prob (F-statistic):             0.0341
Time:                        07:46:38   Log-Likelihood:                -72.705
No. Observations:                  61   AIC:                             173.4
Df Residuals:                      47   BIC:                             203.0
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                   co

In [66]:
from sklearn.preprocessing import StandardScaler

data = results[results['test_loss'].notna()].\
    query('use_batch_norm == True').\
    query('optimizer == "adam"')
X = data[[
    # 'use_batch_norm',
    # 'optimizer',
    'batch_size',
    'include_second_fc_layer',
    # 'activation_type',
    # 'fc_dropout_p',
    'kernel_sizes',
    'out_channels',
]]
# Separate numeric and non-numeric columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
print(f"numeric_cols: {list(numeric_cols)}")
non_numeric_cols = X.select_dtypes(exclude=['float64', 'int64']).columns
print(f"non_numeric_cols: {list(non_numeric_cols)}")

# Standardize numeric columns
scaler = StandardScaler()
X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
# Convert non-numeric columns to dummy variables
X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)
X = X.replace({True: 1, False: 0})
# X = X.to_numpy()
y = data['test_loss']
# standardize the response variable
scaler = StandardScaler()
y = scaler.fit_transform(y.to_numpy().reshape(-1, 1))
# Add a constant to the model (intercept)
X = sm.add_constant(X)
# Fit the logistic regression model
model = sm.OLS(y, X).fit()
# Print the summary
print(model.summary())

numeric_cols: ['batch_size']
non_numeric_cols: ['include_second_fc_layer', 'kernel_sizes', 'out_channels']
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     1.277
Date:                Sun, 07 Jan 2024   Prob (F-statistic):              0.361
Time:                        07:49:16   Log-Likelihood:                -19.144
No. Observations:                  19   AIC:                             58.29
Df Residuals:                       9   BIC:                             67.73
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------

/usr/local/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
